In [ ]:
import numpy as np
import pandas as pd

import os,shutil
import yaml
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import cv2

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

from PIL import Image


In [ ]:
shutil.copytree('../input/yolov50326', '/kaggle/working/yolov5') 


In [ ]:
!pip install -r ./yolov5/requirements.txt

In [ ]:
shutil.copy('../input/vinyolov5xgroupkfold/general.py', '/kaggle/working/yolov5/utils') 

In [ ]:
os.chdir('./yolov5')

In [ ]:
!python ../yolov5/detect.py --weights ../../input/vinyolov5x5foldsweights/fold0-30-best.pt ../../input/vinyolov5x5foldsweights/fold1-30-best.pt ../../input/vinyolov5x5foldsweights/fold4-25-best.pt ../../input/vinyolov5x5foldsweights/fold3-25-best2.pt\
--img 1024\
--conf 0.001\
--iou 0.4\
--source ../../input/vinbigdata-chest-xray-resized-png-1024x1024/test\
--save-txt --save-conf --exist-ok\
--augment

In [ ]:
#!pip install ensemble_boxes

In [ ]:
def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

In [ ]:
from glob import glob

test_df = pd.read_csv('../../input/vinbigdatayolov5sbestweight/test.csv')

image_ids = []
PredictionStrings = []
totalTarget = 0

for file_path in tqdm(glob('runs/detect/exp/labels/*txt')):
    image_id = file_path.split('/')[-1].split('.')[0]
    #print(image_id)
    w, h = test_df.loc[test_df.image_id==image_id,['width', 'height']].values[0]
    f = open(file_path, 'r')
    lines=f.readlines()
    bboxes=''
    for line in lines:
        totalTarget=totalTarget+1
        data = np.array(line.replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
        data = data[:, [0, 5, 1, 2, 3, 4]]
        bbox = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 1).astype(str))
        for idx in range(len(bbox)):
            bbox[idx] = str(int(float(bbox[idx]))) if idx%6!=1 else bbox[idx]
            bboxes=bboxes+str(bbox[idx])+' '
        #print (bboxes,'\n')
    PredictionStrings.append(bboxes)
    image_ids.append(image_id)

In [ ]:
totalTarget=totalTarget/3000
print (totalTarget)

# <a href="yolov5"> Download File </a>

In [ ]:
PredictionStrings

In [ ]:
pred_df = pd.DataFrame({'image_id':image_ids,
                        'PredictionString':PredictionStrings})
sub_df = pd.merge(test_df, pred_df, on = 'image_id', how = 'left').fillna("14 1 0 0 1 1")
sub_df = sub_df[['image_id', 'PredictionString']]
sub_df.head()

In [ ]:
sub_df['PredictionString'].value_counts().iloc[[0]]

In [ ]:
pred_2cls = pd.read_csv('../../input/vinbigdata-2class-prediction/2-cls test pred.csv')
pred = pd.merge(sub_df, pred_2cls, on = 'image_id', how = 'left')
pred.head()

In [ ]:
low_thr  = 0.08
high_thr = 0.95

In [ ]:
def filter_2cls(row, low_thr=low_thr, high_thr=high_thr):
    prob = row['target']
    if prob<low_thr:
        ## Less chance of having any disease
        row['PredictionString'] = '14 1 0 0 1 1'
    elif low_thr<=prob<high_thr:
        ## More change of having any diesease
        row['PredictionString']+=f' 14 {prob} 0 0 1 1'
    elif high_thr<=prob:
        ## Good chance of having any disease so believe in object detection model
        row['PredictionString'] = row['PredictionString']
    else:
        raise ValueError('Prediction must be from [0-1]')
    return row

In [ ]:
sub = pred.apply(filter_2cls, axis=1)
sub.head()

In [ ]:
sub.info()

In [ ]:
sub['PredictionString'].value_counts().iloc[[0]]

In [ ]:
sub[['image_id', 'PredictionString']].to_csv('submission.csv',index = False)